# Deploy Starknet contracts

Import libraries

In [95]:
from starknet_py.net.account.account import Account
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.signer.stark_curve_signer import KeyPair
from starknet_py.net.models.chains import StarknetChainId
from starknet_py.contract import Contract


Create client and signer instances

In [107]:
client = FullNodeClient(node_url='https://starknet-sepolia.infura.io/v3/4deda563b65c4a7c92f5c57b43cd4b29')

deployer_account = Account(
    client=client,
    address="0x07932bfde40a0aa7ece16a2972bd6b4df86b864916d4b5f4fbf2f68aa334ef74",
    key_pair=KeyPair(
        private_key="0x074a5773cc5fe14b48001a4a0249ed45420dc21cd89b072e69079d62456ce04e", 
        public_key="0xdb9c13bd792d489364c86c998068c950d168a8787609baf8ab5e15e7af24e8"
    ),
    chain=StarknetChainId.SEPOLIA,
)

deployer_account

Read accounts file and artifacts from contract

In [108]:
import json

with open("../stjs/accounts.json") as accounts_file:
    accounts = json.load(accounts_file)
    
with open("../src/artifacts/voting.casm") as accounts_file:
    voting_casm = json.load(accounts_file)
    
with open("../src/artifacts/voting.json") as accounts_file:
    voting_json = json.load(accounts_file)
  
voting_json


{'sierra_program': ['0x1',
  '0x6',
  '0x0',
  '0x2',
  '0x8',
  '0x2',
  '0x39a',
  '0x66',
  '0x9b',
  '0x52616e6765436865636b',
  '0x800000000000000100000000000000000000000000000000',
  '0x436f6e7374',
  '0x800000000000000000000000000000000000000000000002',
  '0x1',
  '0x42',
  '0x2',
  '0x6e5f627974657320746f6f20626967',
  '0x1d',
  '0x1000000000000000000000000000000',
  '0x10000000000000000000000000000',
  '0x100000000000000000000000000',
  '0x1000000000000000000000000',
  '0x10000000000000000000000',
  '0x100000000000000000000',
  '0x1000000000000000000',
  '0x10000000000000000',
  '0x100000000000000',
  '0x1000000000000',
  '0x10000000000',
  '0x100000000',
  '0x1000000',
  '0x10000',
  '0x100',
  '0x537472756374',
  '0x800000000000000f00000000000000000000000000000001',
  '0x0',
  '0x2ee1e2b1b89f8c495f200e4956278a4d47395fe262f27b52e5865c9524c08c3',
  '0x456e756d',
  '0x800000000000000700000000000000000000000000000011',
  '0x14cb65c06498f4a8e9db457528e9290f453897bdb216ce18347fff8

Declare contract

In [109]:
# from starknet_py.hash.casm_class_hash import compute_casm_class_hash
# from starknet_py.common import create_casm_class

# declare_result = await Contract.declare_v2(
#     account=deployer_account,
#     compiled_contract=json.dumps(voting_json),
#     compiled_class_hash=compute_casm_class_hash(
#         create_casm_class(json.dumps(voting_casm))
#     ),
#     max_fee=int(1e14),
# )

# await declare_result.wait_for_acceptance()

Deploy contract

In [39]:
from starknet_py.cairo.felt import encode_shortstring

deployer_account.address

3426220078580039383705809181644624414724890574480702587878149058158887497588

In [ ]:
from starknet_py.cairo.felt import encode_shortstring
from starknet_py.common import create_sierra_compiled_contract
from starknet_py.contract import Contract


abi = create_sierra_compiled_contract(
    compiled_contract=json.dumps(voting_json)
).parsed_abi

constructor_args = {
    "voters": list(map(lambda acc: int(acc["address"], 16), accounts["accounts"]))[0:5],
    "yes_init": 223244279740,
    "no_init": 279407245239,
    "n": 753343
}

deploy_result = await Contract.deploy_contract_v1(
    account=deployer_account,
    class_hash="0x0018b117c29169e94b7a99008ffd95d2cd5dbf5087afb0543c350da9d702437c",
    abi=abi,
    constructor_args=constructor_args,
    max_fee=int(1e16),
    cairo_version=1,  # note the `cairo_version` parameter
)

print(deploy_result.deployed_contract.address)

await deploy_result.wait_for_acceptance()
contract = deploy_result.deployed_contract

3093728521938511562198803447950840697241028208090843578170138562168417179718


ValidationError: {'execution_resources': {'data_availability': ['Missing data for required field.']}}

In [112]:
hex(deploy_result.deployed_contract.address)


'0x6d6fd009f0e7493a2c943c84f8b06907fab25977d2416c2c5333d8364785846'

Instance of the contract.

In [ ]:
contract = await Contract.from_address(address=("0x22d55045b9adcf230c2b5c422bf813ae63f8fa1376df738836728e7f390fed7"), provider=deployer_account)


In [74]:
contract.address

433535016476094617525256377778288114263825268424825473898706081032285244718

Call a read method.

In [ ]:
invocation = await contract.functions["get_vote_status"].call()
invocation

((223244279740, 279407245239),)

Get tx receipt.

In [120]:
await client.get_transaction_receipt(tx_hash="0x692350707828ebeecddc49b392c7007dbb9e0b464cc66d7290999516b5a430d")

ValidationError: {'execution_resources': {'data_availability': ['Missing data for required field.']}}